In [1]:
# !pip install ollama gradio

In [ ]:
import random
import gradio as gr
import ollama
from dataclasses import dataclass

# 💬 상수 메시지 정의
MSG_REQUIRE_CODE = "❗ 먼저 비밀 코드를 입력하고 시작해주세요."
MSG_INVALID_CODE = "❗ 비밀 코드는 중복 없는 5자리 숫자이며 첫 숫자는 0이 될 수 없습니다."
MSG_INVALID_GUESS = "❗ 5자리 숫자를 정확히 입력하세요. 예: 12345"

# 🧠 게임 상태 데이터 구조
@dataclass
class GameState:
    ai_code: str
    history: str
    user_attempts: int
    ai_attempts: int
    user_won: bool
    ai_won: bool

# 🔁 초기화 함수
def initialize():
    first_digit = random.choice("123456789")
    ai_code = first_digit + "".join(random.sample([d for d in "0123456789" if d != first_digit], 4))
    return GameState(
        ai_code=ai_code,
        history="",
        user_attempts=0,
        ai_attempts=0,
        user_won=False,
        ai_won=False,
    )

def format_feedback(exact, partial):
    if exact == 0 and partial == 0:
        return "아웃"
    return f"{exact} 스트라이크, {partial} 볼"

def validate_code_input(code):
    return code.isdigit() and len(code) == 5 and code[0] != '0' and len(set(code)) == 5

def validate_guess_input(guess):
    return guess.isdigit() and len(guess) == 5

def compare_codes(guess, target):
    exact = sum(guess[i] == target[i] for i in range(5))
    partial = sum(min(guess.count(d), target.count(d)) for d in set(guess)) - exact
    return exact, partial

def get_ai_guess(history):
    prompt = f"""
[게임 룰]
- 당신과 사용자는 각각 중복 없는 5자리 숫자 코드를 설정합니다 (첫 숫자는 0이 될 수 없음)
- 양쪽은 번갈아가며 서로의 코드를 추리합니다
- 피드백은 '스트라이크(숫자+위치 일치)', '볼(숫자만 일치)', '아웃(불일치)'로 주어집니다
- 선턴이 먼저 정답을 맞힌 경우에만 상대에게 단 한 번의 반격 기회가 주어집니다

당신은 5자리 숫자 코드브레이커 AI입니다. 숫자는 0~9로 이루어지며 중복되지 않습니다.
첫 숫자는 0이 될 수 없습니다.
지금까지의 추측과 피드백을 참고하여 가장 가능성 높은 5자리 숫자를 한 줄로 출력하세요.
지금까지의 기록:
{history}
예시 형식 (형식만 참고, 이 숫자를 그대로 쓰지 마세요): 38429
"""
    for _ in range(5):
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=[{"role": "user", "content": prompt}])
        guess = ''.join(filter(str.isdigit, response["message"]["content"].strip()))
        if validate_code_input(guess):
            return guess
    return "12345"

def process_turn(user_guess, user_code, state: GameState):
    result = state.history
    user_code_display = gr.update(interactive=False)

    if user_code.strip() == "":
        return MSG_REQUIRE_CODE, user_code, state, user_code_display
    if not validate_code_input(user_code):
        return MSG_INVALID_CODE, user_code, state, user_code_display

    if not validate_guess_input(user_guess):
        return MSG_INVALID_GUESS, user_code, state, user_code_display

    state.user_attempts += 1
    user_turn_number = state.user_attempts
    exact_u, partial_u = compare_codes(user_guess, state.ai_code)
    feedback_u = format_feedback(exact_u, partial_u)
    line_u = "\n" + f"[TURN {user_turn_number}] 👤 사용자: {user_guess} → {feedback_u}"
    state.history += "\n" + line_u
    result += "\n" + line_u
    if user_guess == state.ai_code:
        result += "\n" + f"🏆 당신이 AI의 코드를 맞췄습니다! 🎉 (총 시도: {state.user_attempts})"
        state.user_won = True

    ai_guess = get_ai_guess(state.history)
    state.ai_attempts += 1
    ai_turn_number = state.ai_attempts
    exact_a, partial_a = compare_codes(ai_guess, user_code)
    feedback_a = format_feedback(exact_a, partial_a)
    line_a = "\n" + f"[TURN {ai_turn_number}] 🤖 AI: {ai_guess} → {feedback_a}"
    state.history += "\n" + line_a
    result += "\n" + line_a
    if ai_guess == user_code:
        result += "\n" + f"🤖 AI가 당신의 코드를 맞췄습니다! (총 시도: {state.ai_attempts})"
        state.ai_won = True

    return result.strip(), user_code, state, user_code_display

def wrapped_process_turn(user_guess, user_code, *state_vars):
    state = GameState(*state_vars)
    result, user_code, updated_state, user_code_display = process_turn(user_guess, user_code, state)
    return result, user_code, *updated_state.__dict__.values(), user_code_display, gr.update(value="")

def reset_game():
    state = initialize()
    return "", *state.__dict__.values(), gr.update(interactive=True), gr.update(value="")

def ui():
    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 코드브레이커: 사용자 vs AI")

        user_code = gr.Textbox(label="🔐 당신의 비밀 코드 (중복 없는 5자리, 첫 숫자 0 불가)", interactive=True, lines=1)
        user_guess = gr.Textbox(label="🎯 이번 턴의 추측 (5자리 숫자)", lines=1, interactive=True, autofocus=True, placeholder="예: 12345", show_label=True)
        output_box = gr.Markdown(label="📋 결과 출력")
        next_btn = gr.Button("🔄 턴 진행", variant="primary")
        reset_btn = gr.Button("🔁 게임 초기화")

        initial_state = initialize()
        state = [gr.State(v) for v in initial_state.__dict__.values()]

        next_btn.click(fn=wrapped_process_turn,
            inputs=[user_guess, user_code, *state],
            outputs=[output_box, user_code, *state, user_code, user_guess]
        )

        user_guess.submit(
            fn=wrapped_process_turn,
            inputs=[user_guess, user_code, *state],
            outputs=[output_box, user_code, *state, user_code, user_guess]
        )

        reset_btn.click(
            fn=reset_game,
            inputs=[],
            outputs=[user_code, *state, user_code, user_guess]
        )

    return demo

ui().launch(share=True)


* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://030a22a52965196342.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
